# Is there a correlation between font choice and college ranking?
### An frivilous exploration started on a whim 
#### By: Tony Zheng

## Methedology
This exploration is done using the US News university rankings for 2024, collected using a slightly modified version of <a href="https://github.com/kajchang/">kajchang</a>'s <a href="https://github.com/kajchang/USNews-College-Scraper">scraper</a> to collect university website URLs and ranks (outputted to <a href="Scraper/data-detailed.csv">a csv</a>).

In [1]:
from Scraper.main import main as scrape

# each page has 10 schools in it
scrape(n_pages = 15)

Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=1...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=2...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=3...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=4...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=5...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=6...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=7...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=8...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=9...
Fetching https://www.usnews.com/best-colleges/api/search?_sort=rank&_sortDirection=asc&_page=10...
Fetching https://ww

Let's see what it looks like as a DF

In [4]:
import pandas as pd

data = pd.read_csv("Scraper/data-detailed.csv")
data

,institution.displayName,ranking.sortRank,ranking.isTied,Visit School Website
0,Princeton University,1,False,http://www.princeton.edu
1,Massachusetts Institute of Technology,2,False,http://web.mit.edu/
2,Harvard University,3,True,http://www.harvard.edu/
3,Stanford University,3,True,http://www.stanford.edu
4,Yale University,5,False,http://www.yale.edu/
...,...,...,...,...
145,Thomas Jefferson University,142,True,http://www.jefferson.edu/
146,University of Alabama at Birmingham,142,True,https://www.uab.edu
147,University of Cincinnati,142,True,http://www.uc.edu
148,University of the Pacific,142,True,http://www.pacific.edu


... and make sure that no URLs are missing

In [5]:
data.isna().sum()

institution.displayName    0
ranking.sortRank           0
ranking.isTied             0
Visit School Website       0
dtype: int64

In [1]:
from selenium import webdriver

# def get_font_proportion(url):
    # page = requests.get(url)
driver = webdriver.Firefox(executable_path="Scraper/geckodriver.exe")

get_font_proportion("https://www.osu.edu")

ModuleNotFoundError: No module named 'bs4'